In [6]:
# Importing packages

%matplotlib inline
import pandas as pd
import os,cv2
import random as rn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from keras.utils import to_categorical

from keras import layers
from keras.layers import Input
from keras.layers import Dense,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import matplotlib.image as mpimg
#Creating directory path for each dog breed

Maltese_dir = "/content/pics/maltese.jpeg"
Bulldog_dir = "/content/sample_data/bulldog.jpg"
Bull_terrier_dir = "/content/sample_data/bullterrier.jpeg"
Dachshund_dir = "/content/sample_data/dachshund.jpeg"
German_sheperd_dir ="/content/sample_data/germanshepard.jpeg"
Pomeranian_dir = "/content/sample_data/pomerian.jpeg"
Poodle_dir = "/content/sample_data/poodle.jpeg"
X = []
Z = []
IMG_SIZE = 150
# creating a function that returns which type of dog breed(label)

def assign_label(img,dog_type):
    return dog_type
#Function creation- which reads images and labels are classified according to images

def training_data(dog_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,dog_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)

        #resizing images
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))

        X.append(np.array(img))
        Z.append(str(label))
training_data('Maltese',Maltese_dir)
training_data('Bulldog',Bulldog_dir)
training_data('Bull_terrier',Bull_terrier_dir)
training_data('Dachshund',Dachshund_dir)
training_data('German_sheperd',German_sheperd_dir)
training_data('Pomeranian',Pomeranian_dir)
training_data('Poodle',Poodle_dir)
print(X[0])
print(Z)

#Assiging labels via label encoder

label_encoder= LabelEncoder()
Y = label_encoder.fit_transform(Z)

#Representation of Dog breeds and its assigning by the label encoder. so that this can be refered during the output prediction


temp1 = []
for t in Z:
    if t not in temp1:
        temp1.append(t)
breed_name=temp1
print(breed_name)


breed = Y.tolist()
temp = []
for u in breed:
    if u not in temp:
        temp.append(u)
breed=temp
print(breed)

#here the above list represents the label encoding of the dogbreed as specified orderwise


Y = to_categorical(Y,7)
X = np.array(X)
## scaling
X=X/255

#Splitting the Data
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=50)

#Generator creation

augs_gen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=False)

augs_gen.fit(x_train)


#Data Visualization


fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Dog: '+Z[l])

plt.tight_layout()

#Model Building

base_model = InceptionV3(include_top=False,
                  input_shape = (IMG_SIZE,IMG_SIZE,3),
                  weights = 'imagenet')

# Freezing layers
for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(7,activation='softmax'))
model.summary()

#Compiling and Training model

#----------Compile---------------#
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#-----------Training------------#
history = model.fit_generator(
    augs_gen.flow(x_train,y_train,batch_size=16),
    validation_data  = (x_test,y_test),

    epochs = 20,
    verbose = 1,
)


#By training the model, the model give 92% validation accuracy. Here the training accuracy and validation accuracy is not having huge variation. So, the model does not go for overfitting. Hence, model is good fit.

#Visualisation of graph between train accuracy and validation accuracy
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
# Visualisation of graph between train accuracy and validation accuracy
plt.figure(figsize=(12,12))
epochs = list(range(1,21))
plt.plot(epochs,train_acc,label='train_acc',)
plt.plot(epochs,val_acc,label='val_acc')
plt.xlabel("epochs",fontsize=20)
plt.ylabel("accuracy pecentage",fontsize=20)
plt.title('Accuracy graph',fontsize=30)
plt.legend()


#Visualisation of graph between train loss and validation loss


# Visualisation of graph between train loss and validation loss
plt.figure(figsize=(8,8))
plt.plot(epochs,train_loss,label='train_loss')
plt.plot(epochs,val_loss,label='val_loss')
plt.xlabel("epochs",fontsize=20)
plt.ylabel("loss percentage",fontsize=20)
plt.title('loss graph',fontsize=30)
plt.legend()

#Predicting an unknown image

# Predicting unknown dog image
img_url='/content/drive/MyDrive/dogs datasset/36.jpg'
new_image = image.load_img(img_url)
print(type(new_image))

new_image = tf.keras.preprocessing.image.img_to_array(new_image)
print("shape of image :",new_image.shape)
print("Type of image :",type(new_image))

# resizing
new_image = tf.image.resize(new_image,(150,150))

## Scaling
new_image = new_image/255
print('After resizing the image shape :',new_image.shape)
new_image = np.expand_dims(new_image,axis=0)
print("image shape",new_image.shape)

#Prediction Probabilities

model.predict(new_image)

array_num = model.predict(new_image)
num_list = array_num.tolist()
x=num_list[0]
item = max(x)
#search for the item
index = x.index(item)
breed_classes=list(zip(breed,breed_name))
q=0
while q<7:
  if index in breed_classes[q]:
    print('Dog Breed: '+ breed_classes[q][1])
    break
  else:
    q=q+1
img = mpimg.imread(img_url)
imgplot = plt.imshow(img)
plt.show()


NotADirectoryError: [Errno 20] Not a directory: '/content/pics/maltese.jpeg'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')